## Depthwise Separable Convolution Tutorial using Pytorch

In this notebook we are going to implement the depthwise separable convolution procedure utilized in the paper "MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications" by Howard et al.

Lets import the necessary packages.

In [1]:
## Importing necessary packages ##

import torch
import torch.nn as nn

import numpy as np

Its just an experimental process, hence, it wont be that big!

In [2]:
## Implementing the Depthwise Convolution ##
## Here the convolution is done with each filter applied to each channel of the input ##

## The change we implement is with the groups parameter of Conv2d where we choose the input channels as our groups ##
## It hence applies the different filters on different channels of the input ##

depthwise_conv = nn.Conv2d(in_channels = 32 , out_channels = 32 , kernel_size = 3 , stride = 1 , padding = 1 , groups = 32)

Lets try out if we have done it correctly!

In [3]:
## Testing the Depthwise Convolution ##

inp = torch.randn((1 , 32 , 64 , 64))

pred = depthwise_conv(inp)

pred.shape

torch.Size([1, 32, 64, 64])

As expected!!

The number of parameter for this is very small and it helps in efficient implementation.

We can compare it also with a normal convolution.

Lets do that!!

In [4]:
## Simple convolution ##

conv = nn.Conv2d(in_channels = 32 , out_channels = 32 , kernel_size = 3 , stride = 1 , padding = 1 , groups = 1)

In [5]:
print('Depthwise Convolution parameters : ' , depthwise_conv.weight.shape)
print('Normal Convolution parameters : ' , conv.weight.shape)

Depthwise Convolution parameters :  torch.Size([32, 1, 3, 3])
Normal Convolution parameters :  torch.Size([32, 32, 3, 3])


The dependence on input channels is gone!!

Well, the authors of the paper of MobileNet said that the process doesn't stop there and we need to do a pointwise 1 * 1 convolution to complete our approach.

So, lets do that and check the parameters.

In [6]:
## Implementing 1d convolution ##

conv_1d = nn.Conv2d(in_channels = 32 , out_channels = 128 , kernel_size = 1 , stride = 1 )

In [7]:
## Testing our 1d convolution ##

inp1 = torch.randn((1 , 32 , 64 , 64))

pred1 = conv_1d(inp1)

pred1.shape

torch.Size([1, 128, 64, 64])

Perfect.

Now lets check the total weights.

In [8]:
## Depthwise separable convolution weight ##

depthwise_conv_weight = depthwise_conv.weight.shape

depthwise_conv_param = depthwise_conv_weight[0] * depthwise_conv_weight[1] * depthwise_conv_weight[2] * depthwise_conv_weight[3]

conv_1d_weight = conv_1d.weight.shape

conv1d_param = conv_1d_weight[0] * conv_1d_weight[1] * conv_1d_weight[2] * conv_1d_weight[3]

depthwise_total_param = depthwise_conv_param + conv1d_param



## Normal convolution weight ##

conv_weight = conv.weight.shape

conv_param = conv_weight[0] * conv_weight[1] * conv_weight[2] * conv_weight[3]


print('No. of Parameters in Depthwise Separable Convolution : ' , depthwise_total_param)
print('No. of Parameters in Normal Convolution : ' , conv_param)
print('Ratio of Normal Convolution Param to Depthwise Param : ' , conv_param / depthwise_total_param)

No. of Parameters in Depthwise Separable Convolution :  4384
No. of Parameters in Normal Convolution :  9216
Ratio of Normal Convolution Param to Depthwise Param :  2.102189781021898


Normal Convolution operation has twice the number of parameters.

Wow!!

Thats amazing!!

Now lets put it in a class such that we can use it in later cases.

In [9]:
## Depthwise Separable COnvolution ##

class DSConv(nn.Module):
    
    def __init__(self , in_channels , out_channels , kernel_size = 3 , stride = 1 , padding = 1):
        super().__init__()
        self.depthwise_sep_conv = nn.Sequential(nn.Conv2d(in_channels = in_channels , out_channels = in_channels , 
                                                          kernel_size = kernel_size , stride = stride , 
                                                          padding = padding , groups = in_channels),
                                                nn.BatchNorm2d(in_channels),
                                                nn.ReLU(),
                                                nn.Conv2d(in_channels = in_channels , out_channels = out_channels , 
                                                          kernel_size = 1 , stride = stride , padding = 1),
                                                nn.BatchNorm2d(out_channels),
                                                nn.ReLU()
                                               )
    def forward(self , x):
        return self.depthwise_sep_conv(x)

In [10]:
## Testing our module ##

test_ds_conv = DSConv(32 , 128)

pred = test_ds_conv(inp)

pred.shape

torch.Size([1, 128, 66, 66])

Boom!!

Thats all!!

Hopefully it would be easier to build the MobileNet models.